In [187]:
import pandas as pd
import ftfy
import unidecode
import pickle

In [188]:
# handle anomalies/clean data
food_dict = pd.read_csv("D:\\UMass\\Fall 18\\COMPSCI 585 NLP\\Project\\food_dict.csv", encoding="UTF-8")
food_dict['food'] = food_dict.astype(str)
food_dict.drop(['scrape_id'], inplace = True, axis = 1)

food_dict['food'] = food_dict['food'].apply(lambda x: x.lower())
food_dict['food'] = food_dict['food'].apply(lambda x: unidecode.unidecode(x))

In [189]:
# convert to list and save as pickle
food_list = food_dict['food'].tolist()
file_Name = "food_dictionary"
fileObject = open(file_Name,'wb') #wb - raw binary, remember to open the file in rb mode
pickle.dump(food_list,fileObject)   
fileObject.close()

In [190]:
#get data
reviews_df = pd.read_csv("all_reviews.csv")
business_df = pd.read_csv("business.csv")
menus_df = pd.read_csv("all_menus.csv")
bakery_reviews = reviews_df[reviews_df['business_id'] == 1116]['text'].tolist() # tartine bakery and cafe
bakery_menu = menus_df[menus_df['business_id'] == 1116]['menu_items']

In [209]:
bakery_menu

0                                       filtered coffee
1                                  frangipane croissant
2                                           morning bun
3                                 spicy turkey sandwich
4                                          quiche slice
5                                       croque monsieur
6                               double pain au chocolat
7                                          cafe au lait
8                                 cold brew iced coffee
9                                              espresso
10                                            americano
11                                            macchiato
12                                            gibralter
13                                           cappuccino
14                                                latte
15                                        hot chocolate
16                                          almond milk
17                                  matcha almon

In [191]:
from nltk import tokenize
sentences = []
for paragraph in bakery_reviews:
    lines_list = tokenize.sent_tokenize(paragraph)
    sentences.extend(lines_list)
#create a set for fast access!
food_set = set(food_list)

In [220]:
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

# get spans with positions
def spans(txt):
    tokens= word_tokenize(txt)
    offset = 0
    for token in tokens:
        offset = txt.find(token, offset)
        yield token, offset, offset+len(token)
        offset += len(token)

# get mentions
def find_mentions(s, with_stemming = True):
    mentions = []
    for token in spans(s):
        if with_stemming:
            t = stemmer.stem(token[0])
        else:
            t = token[0]
        if t in food_set:
            mentions.append((t, token[1], token[2]))#return the stemmed version but the positions are still from the sentence
    return merge_mentions(mentions)

# merge adjoining mentions to create a multiword mention 
def merge_mentions(intervals):
    out = []
    for i in sorted(intervals, key=lambda i: i[1]):
        if out and (i[1] - out[-1][2]) == 1:
            new_mention = out[-1][0] + " " + i[0]
            new_start = out[-1][1]
            new_end = i[2]
#             print(new_mention)
            out.pop()
            out.append((new_mention, new_start, new_end))
        else:
            out.append(i)
    return out

# get a set of menu items/dishes which we picked out
def get_possible_menu(dict_mentions):
    possible_menu_items = set()
    for mentions in dict_mentions.values():
        if mentions:
#             print(mentions)
            for m in mentions:
                possible_menu_items.add(m[0])
    return possible_menu_items

In [216]:
get_possible_menu(new_dict_mentions)

[[('coconut cream', 41, 54), ('lemon cream', 61, 72), ('orange', 112, 118)], [('coconut cream pie', 41, 59), ('lemon cream tart', 61, 78), ('bun', 92, 96)]]
[[('it', 72, 74)], [('it', 72, 74)]]
[[('banana cream tart', 62, 79)], [('banana cream tart', 62, 79)]]
[[('it', 0, 2), ('pie', 33, 36)], [('it', 0, 2), ('pie', 33, 36)]]
[[('it', 61, 63), ('it', 75, 77)], [('caramel', 39, 50), ('it', 61, 63), ('it', 75, 77)]]
[[], [('croissant', 4, 14)]]


IndexError: list index out of range

### With Stemming (snowball stemmer)

In [193]:
dict_mentions = {}
for s in sentences:
    mentions = find_mentions(s)
    dict_mentions[s] = mentions

In [194]:
dict_mentions

{'if i had more money, i would be awash in coconut cream pies, lemon cream tarts, and morning buns with a hint of orange .': [('coconut cream pie',
   41,
   59),
  ('lemon cream tart', 61, 78),
  ('bun', 92, 96)],
 'a worthwhile treat, and easy to get if you show up a few minutes before it opens at am.': [('it',
   72,
   74)],
 'pretty long waits throughout the day, but you have to try the banana cream tart.': [('banana cream tart',
   62,
   79)],
 "it was the best i've had mission pie comes very close .": [('it', 0, 2),
  ('pie', 33, 36)],
 'the crust is phenomenal and there is a caramelized flavor to it that makes it that much better.': [('caramel',
   39,
   50),
  ('it', 61, 63),
  ('it', 75, 77)],
 'the croissants are huge and the lamination on them are pretty spot on.': [('croissant',
   4,
   14)],
 'i personally thought the croissant was too crunchy though, was looking for a bit of fluff in the middle.': [('croissant',
   25,
   34)],
 'morning bun was also very flaky and fi

#### NOTE: The baseline is heavily dependent on the preprocessing steps and the way the sentence is structured. 
Example sentence : if i had more money, i would be awash in coconut cream pies, lemon cream tarts, and morning buns with a hint of orange . 
Output without stemming : [('coconut', 41, 48),  ('cream', 49, 54), ('lemon', 61, 66),  ('cream', 67, 72), ('orange', 112, 118)] Snowball stemmer : [('coconut', 41, 48), ('cream', 49, 54), ('pies', 55, 59), ('lemon', 61, 66), ('cream', 67, 72),  ('tarts', 73, 78),  ('buns', 92, 96)] Orange which is a common word and found in the food dictionary is missing because it's stem is 'orang' which is not found in the dictionary

In [226]:
menus_with_stemming = get_possible_menu(dict_mentions)
len(menus_with_stemming)

146

###### This is looks pretty good! Some good examples are 'coconut cream tart' and 'cinnamon roll'. However a lot of simple things didn't get picked up. Let's see how many of these are actually on the menu for the restaurant. 

In [228]:
menus_df[(menus_df['menu_items'].isin(menus_with_stemming)) & (menus_df['business_id'] == 1116)].shape[0]
# print("Matching items : ", m_items.shape[0])


10

In [229]:
matching_items = menus_df[menus_df['menu_items'].isin(menu_without_stemming) & (menus_df['business_id'] == 1116)]
print("Matching items: ", matching_items.shape[0])
total_menu_items = menus_df[(menus_df['business_id'] == 1116)]
print("Matching items: ", total_menu_items.shape[0])
print("% matching items - ", (matching_items.shape[0]/total_menu_items.shape[0])*100)
# matching_items
print("Prec :", (matching_items.shape[0]/len(menu_without_stemming))*100)

Matching items:  23
Matching items:  90
% matching items -  25.555555555555554
Prec : 11.6751269035533


### Without stemming 

In [197]:
find_mentions(sentences[0])

[('coconut cream pie', 41, 59), ('lemon cream tart', 61, 78), ('bun', 92, 96)]

In [217]:
new_dict_mentions = {}
for s in sentences:
    mentions = find_mentions(s, with_stemming = False)
    new_dict_mentions[s] = mentions



In [208]:
new_dict_mentions
for key,val in dict_mentions.items():
    if key in new_dict_mentions:
        new_dict_mentions[key] = [new_dict_mentions[key],val]
        
with open("D:\\UMass\\Fall 18\\COMPSCI 585 NLP\\Project\\baseline_no_stemming.txt", "w") as text_file:
        text_file.write(str(new_dict_mentions))

In [212]:
new_dict_mentions

{'if i had more money, i would be awash in coconut cream pies, lemon cream tarts, and morning buns with a hint of orange .': [[('coconut cream',
    41,
    54),
   ('lemon cream', 61, 72),
   ('orange', 112, 118)],
  [('coconut cream pie', 41, 59),
   ('lemon cream tart', 61, 78),
   ('bun', 92, 96)]],
 'a worthwhile treat, and easy to get if you show up a few minutes before it opens at am.': [[('it',
    72,
    74)],
  [('it', 72, 74)]],
 'pretty long waits throughout the day, but you have to try the banana cream tart.': [[('banana cream tart',
    62,
    79)],
  [('banana cream tart', 62, 79)]],
 "it was the best i've had mission pie comes very close .": [[('it', 0, 2),
   ('pie', 33, 36)],
  [('it', 0, 2), ('pie', 33, 36)]],
 'the crust is phenomenal and there is a caramelized flavor to it that makes it that much better.': [[('it',
    61,
    63),
   ('it', 75, 77)],
  [('caramel', 39, 50), ('it', 61, 63), ('it', 75, 77)]],
 'the croissants are huge and the lamination on them ar

In [225]:
menu_without_stemming = get_possible_menu(new_dict_mentions)
matching_items = menus_df[menus_df['menu_items'].isin(menu_without_stemming) & (menus_df['business_id'] == 1116)]
print("Matching items: ", matching_items.shape[0])
total_menu_items = menus_df[(menus_df['business_id'] == 1116)]
print("Matching items: ", total_menu_items.shape[0])
print("% matching items - ", matching_items.shape[0]/total_menu_items.shape[0]*100)
# matching_items
print("Prec :", matching_items.shape[0]/len(menu_without_stemming))

Matching items:  23
Matching items:  90
% matching items -  25.555555555555554
Prec : 0.116751269035533


In [230]:
menus_df[(menus_df['business_id'] == 1116)]

,business_id,menu_items,is_popular
0,1116,filtered coffee,0
1,1116,frangipane croissant,1
2,1116,morning bun,1
3,1116,spicy turkey sandwich,1
4,1116,quiche slice,1
5,1116,croque monsieur,1
6,1116,double pain au chocolat,1
7,1116,cafe au lait,0
8,1116,cold brew iced coffee,0
9,1116,espresso,0


In [232]:
stemmer.stem('cookie')

'cooki'